## Generate a new model:

New Model = Average Model + PCA Deformation


In [3]:
# path_to_your_vtk_file = r'D:\xuhu\humanshape\PCA_model_registration\meanShape.mat'
path_to_your_vtk_file = r'D:\xuhu\humanshape\PCA_model_registration\input\Torso.mat'
path_to_evectors = r'D:\xuhu\humanshape\PCA_model_registration\input\evectors.mat'
path_to_evalues = r'D:\xuhu\humanshape\PCA_model_registration\input\evalues.mat'
path_to_save = r'D:\xuhu\humanshape\PCA_model_registration\deformed_shape.vtk'

In [2]:
from scipy.io import loadmat
import vtk
import numpy as np

mean_model = loadmat(path_to_your_vtk_file)['points']
eigenvalues = loadmat(path_to_evalues)['evalues']
eigenvectors = loadmat(path_to_evectors)['evectors']

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\xuhu\\humanshape\\PCA_model_registration\\Torso.mat'

In [39]:
eigenvalues

array([[5.43754074e+06, 1.00031000e+06, 4.48907981e+05, ...,
        2.65312337e-09, 2.62735867e-09, 2.59993900e-09]])

In [40]:
eigenvectors

array([[ 1.31938530e-04,  2.96730228e-04, -1.38431238e-04, ...,
        -2.27895969e-02, -2.27708416e-02, -2.27210143e-02],
       [-1.81285317e-03, -8.21731769e-04, -1.18085621e-03, ...,
         7.63409105e-05, -4.10634969e-04, -7.68812898e-04],
       [ 5.19278246e-04,  5.67761807e-04, -1.01598407e-04, ...,
         1.18393168e-03,  1.30928598e-03,  1.21962497e-03],
       ...,
       [-3.38031762e-03, -6.34102146e-03, -3.42942589e-03, ...,
         3.63151471e-02,  8.17666310e-03,  2.18881249e-02],
       [-1.03249411e-03,  1.38433965e-03, -1.85990160e-04, ...,
        -8.40456437e-03,  6.50573838e-04, -2.40663076e-02],
       [ 2.43384447e-03,  6.16046841e-04, -5.85437023e-03, ...,
        -2.00187129e-02, -1.11868525e-02, -1.17637010e-02]])

In [29]:
import vtk

def save_points_to_vtk(points, filename):
    num_points = points.shape[0]

    vtk_points = vtk.vtkPoints()
    for i in range(num_points):
        vtk_points.InsertPoint(i, points[i, 0], points[i, 1], points[i, 2])

    vertices = vtk.vtkCellArray()
    for i in range(num_points):
        vertices.InsertNextCell(1)
        vertices.InsertCellPoint(i)

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(vtk_points)
    polydata.SetVerts(vertices)

    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(polydata)
    writer.Write()

In [30]:
test_mean_model = np.array(mean_model)

In [35]:
test_mean_model2 = np.ndarray.tolist(test_mean_model[:,0])
test_mean_model2.extend(np.ndarray.tolist(test_mean_model[:,1]))
test_mean_model2.extend(np.ndarray.tolist(test_mean_model[:,2]))
test_mean_model2 = np.array(test_mean_model2)
test_mean_model2.shape


(4296,)

In [33]:
coef = np.zeros(eigenvalues[0].shape)
mode_id = 1
sigma=0.001
coef[mode_id-1] = eigenvalues[0][mode_id-1]*3*sigma
# coef[mode_id-1] = -eigenvalues[0][mode_id-1]*3*sigma
coef

(4300,)

In [26]:
e_evectors = np.dot(np.diag(coef),eigenvectors)

In [28]:
deformed_model = np.transpose(test_mean_model2) + np.sum(e_evectors, axis=0)
deformed_model.shape

ValueError: operands could not be broadcast together with shapes (4296,) (19347,) 

In [ ]:
# deformed_model = deformed_model.reshape((-1,3))
print(deformed_model.size)
deformed_x = deformed_model[0:int(deformed_model.size/3)-1]
deformed_y = deformed_model[int(deformed_model.size/3): int(2*deformed_model.size/3)-1]
deformed_z = deformed_model[int(2*deformed_model.size/3):-1]
deformed_model = np.transpose([deformed_x, deformed_y, deformed_z])


19347


In [36]:
save_points_to_vtk(mean_model, f"D:\\xuhu\\humanshape\\PCA_model_registration\\Torso.vtk")


In [25]:
import numpy as np

def pca_deformation(mean_model, eigenvectors, coefficients, num_components):
    # Select eigenvectors and coefficients

    mean_model = mean_model.reshape((3*mean_model.shape[0],1))
    selected_eigenvectors = eigenvectors[:num_components]
    selected_coefficients = coefficients[:num_components]

    e_evectors = np.dot(np.diag(eigenvalues[0]),eigenvectors)
    deformed_model = mean_model + np.dot(e_evectors, mean_model)
    deformed_model.reshape((-1, 3))
    # # Calculate the PCA deformation
    # pca_deformation_xyz = np.zeros(mean_model.shape)
    # pca_deformation = np.zeros((1, 19347))
    # for i in range(num_components):
    #     pca_deformation = selected_coefficients[i] * selected_eigenvectors[i]
    
    # Reshape the PCA deformation
    pca_deformation_xyz = pca_deformation.reshape((-1, 3))

    print('pca_deformation\n', pca_deformation)
    print('pca_deformation_xyz\n', pca_deformation_xyz)

    # Generate the new model
    new_model = mean_model + pca_deformation_xyz
    print('new_model\n', new_model)

    return new_model


# Apply PCA deformation and get the new model
new_model = pca_deformation(mean_model, eigenvectors, coefficients, num_components=3)


pca_deformation
 [[ 1.31938530e-05  2.96730228e-05 -1.38431238e-05 ... -2.27895969e-03
  -2.27708416e-03 -2.27210143e-03]]
pca_deformation_xyz
 [[ 1.31938530e-05  2.96730228e-05 -1.38431238e-05]
 [ 7.49833546e-06  3.08650241e-05  1.42035271e-05]
 [ 3.64705978e-05  5.77066314e-06  6.65897991e-06]
 ...
 [-2.24718556e-03 -2.22904634e-03 -2.24094571e-03]
 [-2.25372111e-03 -2.27735618e-03 -2.28184079e-03]
 [-2.27895969e-03 -2.27708416e-03 -2.27210143e-03]]
new_model
 [[-1.32606615e-03  9.05487297e+01  7.99717986e+02]
 [-2.13873609e-01  7.99444405e+01  8.08583813e+02]
 [ 4.47730793e-01  9.94668801e+01  7.94246381e+02]
 ...
 [ 1.63802888e+02  2.36978659e+01  4.74013627e+01]
 [ 1.59368831e+02  4.62183629e+01  4.26744387e+01]
 [ 1.72350609e+02  7.94972618e+01  4.25418361e+01]]
